<a href="https://colab.research.google.com/github/ajayvallabh/PytorchTutorial/blob/main/AFFF_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

!pip install rdkit-pypi

Mounted at /gdrive


In [11]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, MolFromSmiles
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import hyperopt
from hyperopt import STATUS_OK


def positional_encoding(length, depth):

  positions = np.arange(length)[:, np.newaxis]
  depths = np.arange(depth)[np.newaxis, :]/depth

  angle_rates = 1 / (10000**depths)
  angle_rads = positions * angle_rates

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  pos_encoding = pos_encoding[:, :depth]

  return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, d_model):
    super().__init__()
    self.d_model = d_model
    self.dense = tf.keras.layers.Dense(d_model)
    self.pos_encoding = positional_encoding(length=d_model, depth=d_model)

  def call(self, x):
    length = tf.shape(x)[1]
    x = tf.expand_dims(x, -1)
    x = self.dense(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[:length, :]
    return x

class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, FF_num_layers, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential()
    for _ in range(int(FF_num_layers)):
        self.seq.add(tf.keras.layers.Dense(dff, activation='gelu'))
        self.seq.add(tf.keras.layers.Dropout(dropout_rate))
        dff = dff // 2
    self.seq.add(tf.keras.layers.Dense(d_model))
    self.linear = tf.keras.layers.Dense(d_model)
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    seq_output = self.seq(x)
    x = self.add([x, seq_output])
    x = self.linear(x)
    x = self.layer_norm(x)
    return x

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff, dropout_rate)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(d_model=d_model, )

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    x = self.pos_embedding(x)
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)
    return x

class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(1, activation='linear')

  #@tf.function(reduce_retracing=True)
  def call(self, inputs):
    x = inputs
    x = self.encoder(x)
    x = GlobalMaxPooling1D()(x)
    pred = self.final_layer(x)
    return pred

def scale_inputs(df, columns):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[columns])
    return scaled_data, scaler

def preprocess_smiles(smiles_list):
    smiles_list = smiles_list.to_list()
    d_model = 64

    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(smiles_list)
    sequences = tokenizer.texts_to_sequences(smiles_list)
    max_length = max([len(seq) for seq in sequences])
    padded_sequences = pad_sequences(sequences, maxlen=56, padding='post')
    return padded_sequences

def hyp_opt(params):
    print('hyp; params:', params)
    tf.get_logger().setLevel('ERROR')

    # Extract and cast parameters
    num_heads = int(params['num_heads'])
    num_layers = int(params['num_layers'])
    dff = int(params['dff'])
    dropout_rate = float(params['dropout_rate'])
    learning_rate = float(params['learning_rate'])
    batch_size = int(params['batch_size'])
    epochs = int(params['epochs'])
    d_model = int(params['d_model'])

    # Sanity check: ensure multi-head attention compatibility
    if d_model % num_heads != 0:
        return {'loss': float('inf'), 'status': STATUS_OK}

    # Prepare training data
    df_train = train.copy().reset_index(drop=True)
    y = df_train['Removal_rate'].reset_index(drop=True)

    # Encode SMILES sequences
    encoded_smiles = preprocess_smiles(df_train['SMILES'])
    df_encoded = pd.DataFrame(encoded_smiles)

    # Scale numerical inputs
    scaled_inputs, scaler = scale_inputs(df_train, input_vars)
    df_scaled = pd.DataFrame(scaled_inputs)

    # Concatenate features
    X_all = pd.concat([df_scaled, df_encoded], axis=1)

    # Cross-validation setup
    splitter = GroupKFold(n_splits=5)
    cv_rmse = []

    # Iterate folds
    for train_idx, val_idx in splitter.split(X_all, y, groups=df_train['group']):
        # Gather and cast
        X_tr = tf.cast(tf.gather(X_all, train_idx), tf.float32)
        X_val = tf.cast(tf.gather(X_all, val_idx), tf.float32)
        y_tr = y.iloc[train_idx]
        y_val = y.iloc[val_idx]

        # Build functional Model
        inputs = Input(shape=(d_model,), dtype=tf.float32)
        outputs = Transformer(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            dropout_rate=dropout_rate
        )(inputs)
        model = Model(inputs, outputs)

        # Compile & fit
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
        history = model.fit(
            X_tr, y_tr,
            validation_data=(X_val, y_val),
            batch_size=batch_size,
            epochs=epochs,
            verbose=0
        )

        # Evaluate fold
        preds = model.predict(X_val)
        fold_rmse = np.sqrt(mean_squared_error(y_val, preds))
        cv_rmse.append(fold_rmse)

    # Return average RMSE as loss
    return {'loss': np.mean(cv_rmse), 'status': STATUS_OK}

def preprocess_data(df):
      df['SMILES'] = df['SMILES'].str.replace('[()=]', '', regex=True)
      df['charge'] = df.apply(lambda row: calculate_charge(row['pH'], row['pKa'], is_acid=True), axis=1)
      df['charge_product'] = (df['IEP'] - df['pH']) * df['charge']
      df['Init_conc'] = df['Init_conc']/df['Mw']
      df['group'] = df['ref_ID'].map(str) + df['PFAS']

      return df

def calculate_charge(pH, pKa, is_acid=True):
      ratio = 10 ** (pH - pKa)
      fraction_deprotonated = ratio / (1 + ratio)

      if is_acid:
          charge = -fraction_deprotonated
      else:
          charge = 1 - fraction_deprotonated

      return charge

In [12]:
path = '/gdrive/My Drive/Colab Notebooks/AFFF_data_final_20240308.xlsx'
df = pd.read_excel(path)

df.columns = ['ID', 'Membrane', 'MWCO', 'IEP',
              'CA', 'PFAS', 'Mw', 'SMILES',
              'Size', 'log Kow', 'pKa', 'Init_conc',
              'IS', 'Pres', 'pH', 'Removal_rate', 'ref_ID', 'ref']

df = preprocess_data(df)
df = df.dropna(subset=['IEP'])
df.index = range(len(df))

input_vars = ['MWCO', 'CA', 'Size', 'log Kow', 'Init_conc', 'Pres', 'IS', 'charge_product']

train = df[df['ref_ID']!=1]
test = df[df['ref_ID']==1]

In [13]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from tensorflow.keras.utils import pad_sequences
import matplotlib.pyplot as plt

d_model = 64
num_heads_values = [2, 4, 8, 16, 32]
dff_values = [4, 8, 16, 32]
num_layers_values = [1, 2, 3, 4, 5, 6, 7, 8]
batch_size_values = [4, 8, 16, 32, 64]
FF_num_layers_values = [1, 2, 3, 4, 5, 6, 7, 8]

space = {'num_heads': hp.choice('num_heads', num_heads_values),
         'num_layers': hp.choice('num_layers', num_layers_values),
         'dff': hp.choice('dff', dff_values),
         'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.3),
         'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
         'batch_size': hp.choice('batch_size', batch_size_values),
         'epochs': hp.uniformint('epochs', 1, 500),
         'FF_num_layers': hp.choice('FF_num_layers', FF_num_layers_values),
         'd_model': d_model}

trials = Trials()
best_params = fmin(fn=hyp_opt, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

best_params['num_heads'] = num_heads_values[best_params['num_heads']]
best_params['num_layers'] = num_layers_values[best_params['num_layers']]
best_params['dff'] = dff_values[best_params['dff']]
best_params['batch_size'] = batch_size_values[best_params['batch_size']]
best_params['FF_num_layers'] = FF_num_layers_values[best_params['FF_num_layers']]

hyp; params:
{'FF_num_layers': 1, 'batch_size': 8, 'd_model': 64, 'dff': 8, 'dropout_rate': 0.19405025218207847, 'epochs': 382, 'learning_rate': 0.0013587748905997134, 'num_heads': 2, 'num_layers': 1}
1/3 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step

1/3 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 421ms/step

1/3 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step   
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step

1/3 ━━━━━━━━━━━━━━━━━━━━ 1s 748ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 383ms/step

1/3 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step

hyp; params:
{'FF_num_layers': 3, 'batch_size': 4, 'd_model': 64, 'dff': 32, 'dropout_rate': 0.003845893327815264, 'epochs': 140, 'learning_rate': 0.007544619732296216, 'num_heads': 8, 'num_layers': 6}
1/3 ━━━━━━━━━━━━━━━

In [15]:
from datetime import datetime
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
import pandas as pd

# --- 1) Prepare train data ---
# SMILES → sequence encoding
encoded_train = pd.DataFrame(preprocess_smiles(train['SMILES']))
# scale numeric features
train_scaled, scaler = scale_inputs(train, input_vars)
# concat both
X_train_concat = pd.concat([pd.DataFrame(train_scaled), encoded_train], axis=1)

# convert to tensor, cast to float32
X_train_tf = tf.cast(tf.convert_to_tensor(X_train_concat.values), tf.float32)
y_train    = train['Removal_rate']

# --- 2) Build & compile the Model wrapper ---
inputs = Input(shape=(d_model,), dtype=tf.float32)
outputs = Transformer(
    num_layers=best_params['num_layers'],
    d_model=d_model,
    num_heads=best_params['num_heads'],
    dff=best_params['dff'],
    dropout_rate=best_params['dropout_rate']
)(inputs)
model = Model(inputs, outputs)
model.compile(
    optimizer=Adam(learning_rate=best_params['learning_rate']),
    loss='mse'
)

# --- 3) Train on full training set ---
history = model.fit(
    X_train_tf,
    y_train,
    batch_size=best_params['batch_size'],
    epochs=int(best_params['epochs']),
    verbose=1
)

# --- 4) Prepare test data correctly ---
# NOTE: use scaler.transform, not fit_transform
test_scaled = scaler.transform(test[input_vars])
encoded_test = pd.DataFrame(preprocess_smiles(test['SMILES']))
X_test_concat = pd.concat([pd.DataFrame(test_scaled), encoded_test], axis=1)

X_test_tf = tf.cast(tf.convert_to_tensor(X_test_concat.values), tf.float32)
y_test    = test['Removal_rate']

# --- 5) Predict & safely assign back to DataFrames ---
y_pred_test  = model.predict(X_test_tf)
y_pred_train = model.predict(X_train_tf)

test.loc[:,  'y_pred'] = y_pred_test.flatten()
train.loc[:, 'y_pred'] = y_pred_train.flatten()

# define train_predictions for plotting / metrics
train_predictions = y_pred_train.flatten()

Epoch 1/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - loss: 8306.8926
Epoch 2/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 7033.1377
Epoch 3/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 6536.1118
Epoch 4/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 6183.0371
Epoch 5/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 6040.1772
Epoch 6/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5759.6924
Epoch 7/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5420.9053
Epoch 8/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5165.0630
Epoch 9/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4967.9048
Epoch 10/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4700.3818
Epoch 11/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4513.6045
Epoch 12/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4279.7642
Epoch 13/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4061.8779
Epoch 14/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3837.5605
Epoch 15/495
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13m

<ipython-input-15-b3ba8cc94bec>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,  'y_pred'] = y_pred_test.flatten()
<ipython-input-15-b3ba8cc94bec>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, 'y_pred'] = y_pred_train.flatten()


In [17]:
import numpy as np
import sklearn.metrics as metrics
import plotly.express as px
import plotly.graph_objects as go

def plot_predictions(train, test, train_predictions):
    train['mark'] = 'train'
    test['mark'] = 'test'
    total_data = pd.concat([train, test])

    X_title = 'Real removal (%)'
    Y_title = 'Pred removal (%)'
    title_font_size = 43
    tickfont_size = 35
    ticklen = 5
    tickwidth = 2
    title_standoff = 20
    plot_width = 600
    plot_height = 500

    train_pred_mae = mean_absolute_error(train_predictions, train['Removal_rate'])
    train_pred_rmse = np.sqrt(metrics.mean_squared_error(train_predictions, train['Removal_rate']))

    pred_mae = mean_absolute_error(test['y_pred'], test['Removal_rate'])
    pred_rmse = np.sqrt(metrics.mean_squared_error(test['y_pred'], test['Removal_rate']))

    print(f"MAE: {np.round(pred_mae,2)}, RMSE: {np.round(pred_rmse,2)}")
    print(f"Train MAE: {np.round(train_pred_mae,2)}, Train RMSE: {np.round(train_pred_rmse,2)}")


    fig = px.scatter(total_data, x='Removal_rate', y='y_pred', color='mark', width=plot_width, height=plot_height, range_x=[-10, 120], range_y=[-10, 120])

    # Customize layout
    fig.update_layout(
        xaxis=dict(
            dtick=25,
            title_text=X_title,
            title_font={"size": title_font_size},
            tickfont=dict(family='Arial', color='black', size=tickfont_size),
            title_standoff=title_standoff,
            color='black',
            ticks="inside",
            ticklen=ticklen,
            tickwidth=tickwidth,
            title_font_family="Arial"),
        yaxis=dict(
            dtick=25,
            title_text=Y_title,
            title_font={"size": title_font_size},
            tickfont=dict(family='Arial', color='black', size=tickfont_size),
            title_standoff=title_standoff,
            color='black',
            ticks="inside",
            ticklen=ticklen,
            tickwidth=tickwidth,
            title_font_family="Arial"))

    fig.update_layout(plot_bgcolor='rgb(256,256,256)', showlegend=False)

    fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)

    random_x = [0, 100]
    random_y0 = [0, 100]
    fig.add_trace(go.Scatter(x=random_x, y=random_y0, line=dict(color='red', width=2), marker=dict(size=1)))

    fig.update_traces(marker=dict(size=10, opacity=0.65, line=dict(width=0.7)), selector=dict(mode='markers'))

    fig.show()

plot_predictions(train, test, train_predictions)

<ipython-input-17-5525f757cf2b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['mark'] = 'train'
<ipython-input-17-5525f757cf2b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mark'] = 'test'


MAE: 9.02, RMSE: 11.67
Train MAE: 2.39, Train RMSE: 3.55


In [20]:
best_params

{'FF_num_layers': 3,
 'batch_size': 64,
 'dff': 32,
 'dropout_rate': np.float64(0.04650800327517572),
 'epochs': np.float64(495.0),
 'learning_rate': np.float64(0.0007677630673139819),
 'num_heads': 4,
 'num_layers': 8}

In [21]:
def preprocess_smiles(smiles_list):
    smiles_list = smiles_list.to_list()
    d_model = 64

    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(smiles_list)
    sequences = tokenizer.texts_to_sequences(smiles_list)
    max_length = max([len(seq) for seq in sequences])
    padded_sequences = pad_sequences(sequences, maxlen=56, padding='post')
    return padded_sequences, tokenizer